In [1]:
import json
import requests # type: ignore
import pandas as pd # type: ignore
from snowflake.connector import connect # type: ignore
from snowflake.connector.pandas_tools import write_pandas  # type: ignore
from dotenv import load_dotenv # type: ignore
import os

# Data Extraction

In [ ]:
url = 'https://newsapi.org/v2/everything?'  # url of the api

In [ ]:
# def data_extract(file):
#     try: 
#         data = requests.get(url, params = json.loads(open(file, 'r').read())).json()
#         return data
#     except PathError:
#         print('Check your input path')

In [ ]:
def data_extract(file):
    try:
        try:
            file_open = open(file, 'r').read()   # file here is basically the path of your file containing the configurations of the url
            file_open = json.loads(file_open)
            print('File loaded successfully!')
        except:
            print('Error in your configuration file')
        
        file_open['apiKey'] = os.getenv('apiKey')
        print(file_open)
        
        data = requests.get(url, params = file_open).json()
        print('Data extracted successfully!')
        return data
    except:
        print('check your configuration path')

In [ ]:
extracted_data = data_extract(r'C:\Users\JayashreeArunan\Desktop\Python\Data_extraction\configuration\api_config.json')

In [ ]:
df = pd.DataFrame(extracted_data['articles'])
df.head(5)

In [ ]:
df.insert(0, 'source_id', pd.json_normalize(df['source']).get('id'))
df.insert(1, 'source_name', pd.json_normalize(df['source']).get('name'))

In [ ]:
df = df.drop('source', axis=1)

In [ ]:
df.head(5)

In [ ]:
df.to_csv('Data.csv')

# Snowflake Connection

### Using json file method and environment variable method

In [ ]:
def snowflake_credentials(sf_config):
    try:
        sf_file_open = open(sf_config, 'r').read()
        sf_file_open = json.loads(sf_file_open)
    except:
        print('Error in file path')

    sf_connect = connect(
        # user = sf_file_open['user'],
        # password = sf_file_open['password'],
        
        user = os.environ.get('sf_user'),
        password = os.environ.get('sf_password'),
        account = sf_file_open['account'],
        role = sf_file_open['role'],
        warehouse = sf_file_open['warehouse'],
        database = sf_file_open['database'],
        schema = sf_file_open['schema']
    )
    
    # sf_connect = connect(
    #     user = sf_file_open.get('user'),
    #     password = sf_file_open.get('password'),
    #     account = sf_file_open.get('account'),
    #     role = sf_file_open.get('role'),
    #     warehouse = sf_file_open.get('warehouse'),
    #     database = sf_file_open.get('database'),
    #     schema = sf_file_open.get('schema')
    # )
    return sf_connect

In [ ]:
# using all the credentials from a json file
# write_pandas(snowflake_credentials(r'C:\Users\JayashreeArunan\Desktop\Python\Data_extraction\configuration\sf_config.json'), df, 'NEW_TABLE', auto_create_table=True, overwrite=True)

In [ ]:
# using the username and password stored in the system environment varibles and rest of the credentials from the json file
# write_pandas(snowflake_credentials(r'C:\Users\JayashreeArunan\Desktop\Python\Data_extraction\configuration\sf_config.json'), df, 'NEW_TABLE1', auto_create_table=True, overwrite=True)

### Using .env method

In [ ]:
load_dotenv()

In [ ]:
def snowflake_credentials():
    try:
        sf_connect = connect(
            user = os.getenv('user'),
            password = os.getenv('password'),
            account = os.getenv('account'),
            role = os.getenv('role'),
            warehouse = os.getenv('warehouse'),
            database = os.getenv('database'),
            schema = os.getenv('schema')
        )
        print('Connection successful')
    except:
        print('Error in credentials')
    return sf_connect

In [ ]:
write_pandas(snowflake_credentials(), df, 'NEW_TRY', auto_create_table=True, overwrite=True)